In [3]:
import cv2, numpy as np, pandas as pd

world_pts = np.float32([
  [10,0,10],[90,0,10],[10,0,90],[90,0,90],
  [0,10,10],[0,90,10],[0,10,90],[0,90,90],
  [10,10,100],[90,10,100],[10,90,100],[90,90,100]
])

# === 2. ここに対応する (u,v) pxを記入===
img_pts = np.float32([
  [522,312],[784,300],[540,128],[803,120],
  [478,345],[736,330],[493,165],[753,155],
  [545, 92],[807, 84],[557, 11],[820,  6]
])

h, w = 1520, 2288     # 画像サイズ(px)を入力
K0 = np.array([[w,0,w/2],[0,w, h/2],[0,0,1]], dtype=np.float64)  # 粗い初期値

ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
    [world_pts], [img_pts], (w,h),
    K0, None,
    flags=cv2.CALIB_USE_INTRINSIC_GUESS |
          cv2.CALIB_ZERO_TANGENT_DIST |
          cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5)

# CSV に保存して終了
pd.DataFrame(np.hstack([K, np.zeros((3,1))])).to_csv("K.csv", index=False, header=False)
pd.DataFrame(cv2.Rodrigues(rvecs[0])[0]).to_csv("R.csv", index=False, header=False)
pd.DataFrame(tvecs[0]).to_csv("t.csv", index=False, header=False)
print("RMS reprojection error:", ret)


RMS reprojection error: 67.76761020733406


In [4]:
import cv2, csv

IMG = 'box.jpg'        # キャリブ画像
OUT = 'coords.csv'     # 出力先
points = []            # 応答配列

def on_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        idx = len(points)+1
        points.append((idx, x, y))
        print(f'{idx}: ({x}, {y})')
        cv2.circle(img, (x, y), 4, (0, 0, 255), -1)
        cv2.imshow('pick', img)
        if idx == 12:           # 12点揃ったら保存
            with open(OUT, 'w', newline='') as f:
                csv.writer(f).writerows(points)
            print('saved ->', OUT)
            cv2.destroyAllWindows()

img = cv2.imread(IMG)
cv2.namedWindow('pick'), cv2.setMouseCallback('pick', on_click)
cv2.imshow('pick', img); cv2.waitKey(0)


[ WARN:0@149.327] global loadsave.cpp:268 findDecoder imread_('box.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/highgui/src/window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


: 